In [ ]:
datapath =  ["dataset_1.csv", "dataset_2.csv"]

In [ ]:
import pandas as pd

data = pd.read_csv(datapath)

In [ ]:
filtered = data[data["AGE_FLAG_NM"] == "청소년"].filter(items=["MESURE_AGE_CO", "COAW_FLAG_NM", "MVM_PRSCRPTN_CN", "SEXDSTN_FLAG_CD"])

In [ ]:
exercises = {}

for line in filtered['MVM_PRSCRPTN_CN'].values:
    if not ("본운동:" in line):
        continue
    
    for exercise in line.split("본운동:")[1].split("/")[0].split(","):
        if ("루틴프로그램" in exercise): continue
        
        exercise = exercise.strip()
        if exercise not in exercises:
            exercises[exercise] = 1
        exercises[exercise] += 1

In [ ]:
pd.DataFrame(exercises.items(), columns=["운동", "횟수"]).sort_values(by="횟수", ascending=False)[0:9]

In [ ]:
def one_hot_encoder(exercise:str):
    if exercise == "조깅":
        return [1, 0, 0, 0, 0, 0, 0, 0, 0]
    
    if exercise == "줄넘기 운동":
        return [0, 1, 0, 0, 0, 0, 0, 0, 0]
    
    if exercise == "걷기":
        return [0, 0, 1, 0, 0, 0, 0, 0, 0]
    
    if exercise == "버피 테스트	":
        return [0, 0, 0, 1, 0, 0, 0, 0, 0]
    
    if exercise == "자전거타기":
        return [0, 0, 0, 0, 1, 0, 0, 0, 0]
    
    if exercise == "앉았다 일어서기":
        return [0, 0, 0, 0, 0, 1, 0, 0, 0]
    
    if exercise == "몸통 들어올리기":
        return [0, 0, 0, 0, 0, 0, 1, 0, 0]
    
    if exercise == "수영":
        return [0, 0, 0, 0, 0, 0, 0, 1, 0]
    
    if exercise == "팔굽혀 펴기":
        return [0, 0, 0, 0, 0, 0, 0, 0, 1]
    